# Summarizing Data Post QC

Author: Zan Koenig

## Index
1. [Set Default Paths](#1.-Set-Default-Paths)
2. [Read in Pre-QC Dataset and Apply Quality Control Filters](#2.-Read-in-Pre-QC-Dataset-and-Apply-Quality-Control-Filters)
3. [Organize the Dataset](#3.-Organize-the-Dataset)
4. [Annotate Table with Relatedness Information](#4.-Annotate-Table-with-Relatedness-Information)
5. [Calculate Statistics per Population](#5.-Calculate-Statistics-per-Population)
6. [Format Table for Exporting](#6.-Format-Table-for-Exporting)
7. [Export Final Table](#7.-Export-Final-Table)

# General Overview:

The purpose of this script is to format and write out a tsv which will be used to create plots and summaries of the post-QC dataset in R.

**This script contains information on how to:**
- Select specific columns from a matrix table (shortened as mt)
- Annotate filter flags onto a matrix table
- Join the columns of two matrix tables
- Join two tables
- Group a matrix table by region, population 
- Use `hl.agg.stats` to calculate statics for a metric within a population
- Count the number of samples where a filter flag equals `True`  

In [1]:
import hail as hl

# Functions from gnomAD library to apply genotype filters   
from gnomad.utils.filtering import filter_to_adj

In [2]:
# Initializing Hail 
hl.init()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SPARKMONITOR_LISTENER: Started SparkListener for Jupyter Notebook
SPARKMONITOR_LISTENER: Port obtained from environment: 47137
SPARKMONITOR_LISTENER: Application Started: application_1701358420828_0004 ...Start Time: 1701374900947


Running on Apache Spark version 3.3.0
SparkUI available at http://znk-m.c.diverse-pop-seq-ref.internal:43895
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.122-be9d88a80695
LOGGING: writing to /home/hail/hail-20231130-2008-0.2.122-be9d88a80695.log


In [ ]:
# Allow output scrolling in Jupyter nb viewer for cells with long outputs 

from IPython.core.display import HTML
css = open('format.css').read()
HTML('<style>{}</style>'.format(css))

# 1. Set Default Paths

These default paths can be edited by users as needed. It is recommended to run these tutorials without writing out datasets.

**By default, all of the dataset write out sections are shown as markdown cells. If you would like to write out your own dataset, you can copy the code and paste it into a new code cell. Don't forget to change the paths in the following cell accordingly.** 

[Back to Index](#Index)

In [15]:
# Path for HGDP+1kGP dataset prior to applying gnomAD QC filters
pre_qc_path = 'gs://gcp-public-data--gnomad/release/3.1.2/mt/genomes/gnomad.genomes.v3.1.2.hgdp_1kg_subset_dense.mt'

# PCA outliers file 
outliers_path = 'gs://gcp-public-data--gnomad/release/3.1/secondary_analyses/hgdp_1kg_v2/pca/pca_outliers.txt'

# Path for gnomAD's HGDP+1kGP metadata with updated population labels
metadata_path = 'gs://gcp-public-data--gnomad/release/3.1/secondary_analyses/hgdp_1kg_v2/metadata_and_qc/gnomad_meta_updated.tsv'

# Paths to related and unrelated Matrix Tables (without outliers) written out in Notebook 2: PCA and Ancestry Analyses
unrelateds_path = 'gs://gcp-public-data--gnomad/release/3.1/secondary_analyses/hgdp_1kg_v2/pca_results/unrelateds_without_outliers.mt'
relateds_path = 'gs://gcp-public-data--gnomad/release/3.1/secondary_analyses/hgdp_1kg_v2/pca_results/relateds_without_outliers.mt'

# Path for final output table in tsv format
final_table_path = 'gs://gcp-public-data--gnomad/release/3.1/secondary_analyses/hgdp_1kg_v2/metadata_and_qc/post_qc_summary.tsv'

# 2. Read in Pre-QC Dataset and Apply Quality Control Filters

Since the post-QC mt was not written out, we run the same function as tutorial notebook 1 and 2 to apply the quality control filters to the pre-QC dataset.

**To avoid errors, make sure to run the next two cells before running any code that includes the post-QC dataset.**

**If running the cell below results in an error, double check that you used the  `--packages gnomad` argument when starting your cluster.**  

- See the tutorials [README](https://github.com/atgu/hgdp_tgp/tree/master/tutorials#readme) for more information on how to start a cluster.

<br>
<details><summary> For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<ul>
<li><a href="https://hail.is/docs/0.2/methods/impex.html#hail.methods.read_matrix_table"> More on  <i> read_matrix_table() </i></a></li>

<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.count"> More on  <i> count() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/linalg/hail.linalg.BlockMatrix.html#hail.linalg.BlockMatrix.filter_cols"> More on  <i> filter_cols() </i></a></li>

<li><a href="https://hail.is/docs/0.2/linalg/hail.linalg.BlockMatrix.html#hail.linalg.BlockMatrix.filter_rows"> More on  <i> filter_rows() </i></a></li>
</ul>
</details>

[Back to Index](#Index)

In [4]:
# Set up function to:
# apply gnomAD's sample, variant and genotype QC filters
# remove two contaminated samples identified using CHARR - https://pubmed.ncbi.nlm.nih.gov/37425834/
# remove the gnomAD sample that's added for QC purposes
# only keep the variants which are found in the samples that are left 
# add gnomAD's HGDP+1kGP metadata with the updated population labels as a column field 

def run_qc(mt):
    
    ## Apply sample QC filters to dataset 
    # This filters to only samples that passed gnomAD's sample QC hard filters  
    mt = mt.filter_cols(~mt.gnomad_sample_filters.hard_filtered) # removed 31 samples
    
    ## Apply variant QC filters to dataset
    # This subsets to only PASS variants - those which passed gnomAD's variant QC
    # PASS variants have an entry in the filters field 
    mt = mt.filter_rows(hl.len(mt.filters) != 0, keep=False)
    
    # Remove the two contaminated samples identified by CHARR and 'CHMI_CHMI3_WGS2'
    contaminated_samples = {'HGDP01371', 'LP6005441-DNA_A09'}
    contaminated_samples_list = hl.literal(contaminated_samples)
    mt = mt.filter_cols(~contaminated_samples_list.contains(mt['s']))
    
    # CHMI_CHMI3_WGS2 is a sample added by gnomAD for QC purposes and has no metadata info 
    mt = mt.filter_cols(mt.s == 'CHMI_CHMI3_WGS2', keep = False)

    # Only keep the variants which are found in the samples that are left 
    mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))
    
    # Read in and add the metadata with the updated population labels as a column field 
    metadata = hl.import_table(metadata_path, impute = True, key = 's') 
    mt = mt.annotate_cols(meta_updated = metadata[mt.s])
    
    ## Apply genotype QC filters to the dataset
    # This is done using a function imported from gnomAD and is the last step in the QC process
    mt = filter_to_adj(mt)

    return mt

In [5]:
# Read in the HGDP+1kGP pre-QC mt
pre_qc_mt = hl.read_matrix_table(pre_qc_path)

# Run QC 
mt = run_qc(pre_qc_mt)

2023-11-30 20:08:47.863 Hail: INFO: Reading table to impute column types 1) / 1]
2023-11-30 20:08:52.892 Hail: INFO: Loading <StructExpression of type struct{s: str, `project_meta.sample_id`: str, `project_meta.research_project_key`: str, `project_meta.seq_project`: str, `project_meta.ccdg_alternate_sample_id`: str, `project_meta.ccdg_gender`: str, `project_meta.ccdg_center`: str, `project_meta.ccdg_study`: str, `project_meta.cram_path`: str, `project_meta.project_id`: str, `project_meta.v2_age`: str, `project_meta.v2_sex`: str, `project_meta.v2_hard_filters`: str, `project_meta.v2_perm_filters`: str, `project_meta.v2_pop_platform_filters`: str, `project_meta.v2_related`: str, `project_meta.v2_data_type`: str, `project_meta.v2_product`: str, `project_meta.v2_product_simplified`: str, `project_meta.v2_qc_platform`: str, `project_meta.v2_project_id`: str, `project_meta.v2_project_description`: str, `project_meta.v2_internal`: str, `project_meta.v2_investigator`: str, `project_meta.v2_kno

# 3. Organize the Dataset

In this section we are conducting some steps to get the post-QC dataset ready for downstream analyses. We first create a table with only sample data and then select only the columns we need for the table we will write out. 

Below are descriptions of some fields used in the following analyses:
- mt.meta_updated['hgdp_tgp_meta.Genetic.region'] = Geographic Region (7 in total)
- mt.meta_updated['population'] = Population within each geographic region (80 in total)
- mt.sv_counts.CNV = Number of Copy Number Variants (varies per genome)

<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<br>
<li><a href="https://hail.is/docs/0.2/utils/index.html#hail.utils.hadoop_open"> More on  <i> hadoop_open() </i></a></li>

<li><a href="https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc"> More on  <i> sample_qc() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.cols"> More on  <i> cols() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.select"> More on  <i> select() </i></a></li>    
    
<li><a href="https://hail.is/docs/0.2/methods/impex.html#hail.methods.read_table"> More on  <i> read_table() </i></a></li>

<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.count"> More on  <i> count() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.checkpoint"> More on  <i> checkpoint() </i></a></li>
     
</details>

[Back to Index](#Index)

In [6]:
# Remove PCA outliers from the dataset

# Use hl.hadoop_open to read in the PCA outliers file into Hail from Google Cloud Storage
with hl.utils.hadoop_open(outliers_path) as file:
    outliers = [line.rstrip('\n') for line in file]

# Use hl.literal to convert the outliers list from a python object to a Hail expression so that it can be used to filter out samples
outliers_list = hl.literal(outliers)
 
# Keep the samples which are not contained in the pca outlier list
mt_without_outliers = mt.filter_cols(~outliers_list.contains(mt['s']))

mt_without_outliers.count() 

(159339147, 4094)

In [29]:
# Grab the column fields of the Matrix Table 
mt_col_table = mt_without_outliers.cols()

# Write a col table with only the columns needed for table 1
mt_col_table = mt_col_table.select(mt_col_table.meta_updated['hgdp_tgp_meta.Genetic.region'],
                             mt_col_table.meta_updated.population,
                             mt_col_table.sample_qc.n_snp, 
                             mt_col_table.bam_metrics.mean_coverage)

# Validity check - there should be 4094 samples 
mt_col_table.count()

4094

# 4. Annotate Table with Relatedness Information

Relatedness information is added to the dataset so we can filter out related individuals. 

<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
    
<br>
<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_cols"> More on  <i> annotate_cols() </i></a></li>

<li><a href="https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.counter"> More on  <i> counter() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.union_cols"> More on  <i> union_cols() </i></a></li>
    
</details>

[Back to Index](#Index)

In [30]:
# Need to get number of unrelateds annotated to the table
# Reading in the metadata file with the updated population labels
metadata = hl.import_table(metadata_path, impute = True, key = 's') 

# Read in the unrelated and related matrix tables which were written out in Notebook 2: PCA and Ancestry Analyses
unrelateds = hl.read_matrix_table(unrelateds_path)
relateds = hl.read_matrix_table(relateds_path)

# Annotating the mts with the metadata with updated population labels
unrelateds = unrelateds.annotate_cols(meta_updated = metadata[unrelateds.s])
relateds = relateds.annotate_cols(meta_updated = metadata[relateds.s])

# Annotate both the unrelated and the related tables with a flag named 'unrelated' 
# Set the unrelated flag to 'True' for those in the unrelated dataset and 'False' for those in the related dataset
unrelateds = unrelateds.annotate_cols(unrelated = True)
relateds = relateds.annotate_cols(unrelated = False)

# Use hl.cols() to obtain two tables with only the columns from the unrelated and related mts
unrelateds_cols = unrelateds.cols()
relateds_cols = relateds.cols()

# Validity check 
print(unrelateds_cols.count(), relateds_cols.count()) # 3378 unrelated and 718 related samples = 4096 total samples 

2023-11-30 21:59:31.406 Hail: INFO: Reading table to impute column types
2023-11-30 21:59:34.428 Hail: INFO: Loading <StructExpression of type struct{s: str, `project_meta.sample_id`: str, `project_meta.research_project_key`: str, `project_meta.seq_project`: str, `project_meta.ccdg_alternate_sample_id`: str, `project_meta.ccdg_gender`: str, `project_meta.ccdg_center`: str, `project_meta.ccdg_study`: str, `project_meta.cram_path`: str, `project_meta.project_id`: str, `project_meta.v2_age`: str, `project_meta.v2_sex`: str, `project_meta.v2_hard_filters`: str, `project_meta.v2_perm_filters`: str, `project_meta.v2_pop_platform_filters`: str, `project_meta.v2_related`: str, `project_meta.v2_data_type`: str, `project_meta.v2_product`: str, `project_meta.v2_product_simplified`: str, `project_meta.v2_qc_platform`: str, `project_meta.v2_project_id`: str, `project_meta.v2_project_description`: str, `project_meta.v2_internal`: str, `project_meta.v2_investigator`: str, `project_meta.v2_known_pop`:

3378 718


In [31]:
unrelateds_count = unrelateds_cols.aggregate(hl.agg.counter(unrelateds_cols.meta_updated.population))
relateds_count = relateds_cols.aggregate(hl.agg.counter(relateds_cols.meta_updated.population))

# Validity check - print out the number of unrelated and related individuals per population 
print(f"Number of unrelated individuals per population: \
{unrelateds_count}\n\nNumber of related individuals per population: {relateds_count}")

Number of unrelated individuals per population: {'ACB': 94, 'ASW': 52, 'Adygei': 17, 'BEB': 99, 'Balochi': 23, 'BantuKenya': 10, 'BantuSouthAfrica': 8, 'Basque': 23, 'Bedouin': 42, 'BergamoItalian': 11, 'Biaka': 22, 'Bougainville': 11, 'Brahui': 23, 'Burusho': 24, 'CDX': 88, 'CEU': 120, 'CHB': 103, 'CHS': 103, 'CLM': 94, 'Cambodian': 9, 'Colombian': 3, 'Dai': 9, 'Daur': 9, 'Druze': 35, 'ESN': 103, 'FIN': 98, 'French': 27, 'GBR': 87, 'GIH': 100, 'GWD': 116, 'Han': 33, 'Hazara': 16, 'Hezhen': 8, 'IBS': 104, 'ITU': 102, 'JPT': 102, 'Japanese': 29, 'KHV': 101, 'Kalash': 21, 'Karitiana': 10, 'LWK': 91, 'Lahu': 5, 'MSL': 83, 'MXL': 63, 'Makrani': 22, 'Mandenka': 20, 'Maya': 19, 'Mbuti': 12, 'Miao': 10, 'Mongolian': 10, 'Mozabite': 25, 'Naxi': 8, 'NorthernHan': 10, 'Orcadian': 14, 'Oroqen': 8, 'PEL': 85, 'PJL': 97, 'PUR': 104, 'Palestinian': 38, 'PapuanHighlands': 9, 'PapuanSepik': 8, 'Pathan': 24, 'Pima': 11, 'Russian': 25, 'STU': 98, 'San': 6, 'Sardinian': 27, 'She': 9, 'Sindhi': 22, 'Surui

In [33]:
# Join the columns of the unrelated and related datasets
mt_combined = unrelateds.union_cols(relateds)

# Validity check - count the number of unrelateds (True values) in the mt to make sure it is as expected
# 3378 True and 718 False
print(f'Count for relateds/unrelateds pre-QC: {mt_combined.aggregate_cols(hl.agg.counter(mt_combined.unrelated))}') 
# Create a table with only the columns from the mt containing related information
# This is done since the final output will be a tsv and thus must be in table format
# Being a table of columns allows it to be annotated onto the existing mt_col_table as shown below
mt_combined_col_table = mt_combined.cols()

# Annotate the relatedness information onto the column table
mt_col_table = mt_col_table.annotate(unrelated = mt_combined_col_table[mt_col_table.s].unrelated)

# Validity check - count the number of unrelateds (True values) in the mt to make sure it is as expected (post-QC)
# 3376 True and 718 False
print(f'Count for relateds/unrelated post-QC: {mt_col_table.aggregate(hl.agg.counter(mt_col_table.unrelated))}')

Count for relateds/unrelateds pre-QC:{False: 718, True: 3378}


2023-11-30 22:07:14.480 Hail: INFO: Coerced sorted dataset          (0 + 4) / 4]
2023-11-30 22:07:15.329 Hail: INFO: Coerced sorted dataset


Count for relateds/unrelated post-QC: {False: 718, True: 3376}


In [34]:
mt_col_table.show(5)

2023-11-30 22:07:51.219 Hail: INFO: Coerced sorted dataset
2023-11-30 22:07:52.057 Hail: INFO: Coerced sorted dataset
2023-11-30 22:07:52.703 Hail: INFO: Coerced sorted dataset


,,,,,,,
s,hgdp_tgp_meta.Genetic.region,population,n_snp,mean_coverage,CNV,all,unrelated
str,str,str,int64,float64,int32,int32,bool
"""HG00096""","""EUR""","""GBR""",2566022,3.29e+01,511,8011,True
"""HG00097""","""EUR""","""GBR""",2569966,3.15e+01,506,8114,True
"""HG00099""","""EUR""","""GBR""",2567942,3.64e+01,501,8147,True
"""HG00100""","""EUR""","""GBR""",2576696,3.02e+01,488,8245,True
"""HG00101""","""EUR""","""GBR""",2565175,3.28e+01,470,8042,True


# 5. Calculate Statistics per Population

In this section, we will be using `hl.agg.stats()` which calculates the following metrics for a given expression:
- min
- max
- mean
- standard deviation
- number of non-missing records
- sum
 
Using `hl.group_by()`, we calculate these statistics for each of the 78 populations in this dataset.
We also count the number of related samples within each populations by using `hl.agg.count_where()` and count the number of times the field denoting if samples are related or not is True.

<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<br>
<li><a href="https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.stats"> More on  <i> stats() </i></a></li>

<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.group_by"> More on  <i> group_by() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.count_where"> More on  <i> count_where() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.expr.TupleExpression.html#hail.expr.TupleExpression.show"> More on <i>show()</i></a></li>
     
</details>

[Back to Index](#Index)

In [36]:
mt_col_table.describe()

----------------------------------------
Global fields:
    'global_annotation_descriptions': struct {
        gnomad_sex_imputation_ploidy_cutoffs: struct {
            Description: str
        }, 
        gnomad_population_inference_pca_metrics: struct {
            Description: str
        }, 
        sample_hard_filter_cutoffs: struct {
            Description: str
        }, 
        gnomad_sample_qc_metric_outlier_cutoffs: struct {
            Description: str
        }, 
        gnomad_age_distribution: struct {
            Description: str, 
            sub_globals: struct {
                bin_edges: struct {
                    Description: str
                }, 
                bin_freq: struct {
                    Description: str
                }, 
                n_smaller: struct {
                    Description: str
                }, 
                n_larger: struct {
                    Description: str
                }
            }
        }, 
        hgdp_tgp

In [38]:
# Calculate stats per population for each metric grouped by genetic region and population
table = mt_col_table.group_by(
    mt_col_table['hgdp_tgp_meta.Genetic.region'], mt_col_table.population).aggregate(
    n_snp_stats = hl.agg.stats(mt_col_table.n_snp),
    cov_stats = hl.agg.stats(mt_col_table.mean_coverage),
    n_unrelated = hl.agg.count_where(mt_col_table.unrelated == True))

In [39]:
# Check that each of the table fields contain what we'd expect
table.show()

2023-11-30 22:11:00.840 Hail: INFO: Coerced sorted dataset
2023-11-30 22:11:01.520 Hail: INFO: Coerced sorted dataset
2023-11-30 22:11:02.195 Hail: INFO: Coerced sorted dataset
2023-11-30 22:11:03.414 Hail: INFO: Ordering unsorted dataset with network shuffle


+------------------------------+--------------------+------------------+
| hgdp_tgp_meta.Genetic.region | population         | n_snp_stats.mean |
+------------------------------+--------------------+------------------+
| str                          | str                |          float64 |
+------------------------------+--------------------+------------------+
| "AFR"                        | "ACB"              |         3.14e+06 |
| "AFR"                        | "ASW"              |         3.08e+06 |
| "AFR"                        | "BantuKenya"       |         3.18e+06 |
| "AFR"                        | "BantuSouthAfrica" |         3.27e+06 |
| "AFR"                        | "Biaka"            |         3.41e+06 |
| "AFR"                        | "ESN"              |         3.20e+06 |
| "AFR"                        | "GWD"              |         3.19e+06 |
| "AFR"                        | "LWK"              |         3.19e+06 |
| "AFR"                        | "MSL"              |         3.23e+06 |
| "AFR"                        | "Mandenka"         |         3.18e+06 |
+------------------------------+--------------------+------------------+

+-------------------+-----------------+-----------------+---------------+
| n_snp_stats.stdev | n_snp_stats.min | n_snp_stats.max | n_snp_stats.n |
+-------------------+-----------------+-----------------+---------------+
|           float64 |         float64 |         float64 |         int64 |
+-------------------+-----------------+-----------------+---------------+
|          4.63e+04 |        2.92e+06 |        3.20e+06 |           114 |
|          6.23e+04 |        2.88e+06 |        3.18e+06 |            71 |
|          2.01e+04 |        3.14e+06 |        3.21e+06 |            12 |
|          4.15e+04 |        3.21e+06 |        3.35e+06 |             8 |
|          9.15e+03 |        3.39e+06 |        3.43e+06 |            26 |
|          8.46e+03 |        3.18e+06 |        3.23e+06 |           148 |
|          1.19e+04 |        3.15e+06 |        3.22e+06 |           176 |
|          9.91e+03 |        3.15e+06 |        3.22e+06 |            97 |
|          8.52e+03 |        3.20e+06 |        3.26e+06 |            98 |
|          9.50e+03 |        3.16e+06 |        3.21e+06 |            23 |
+-------------------+-----------------+-----------------+---------------+

+-----------------+----------------+-----------------+---------------+
| n_snp_stats.sum | cov_stats.mean | cov_stats.stdev | cov_stats.min |
+-----------------+----------------+-----------------+---------------+
|         float64 |        float64 |         float64 |       float64 |
+-----------------+----------------+-----------------+---------------+
|        3.58e+08 |       3.17e+01 |        2.48e+00 |      2.84e+01 |
|        2.18e+08 |       3.23e+01 |        3.36e+00 |      2.75e+01 |
|        3.82e+07 |       3.19e+01 |        3.88e+00 |      2.91e+01 |
|        2.62e+07 |       3.89e+01 |        1.03e+01 |      3.05e+01 |
|        8.86e+07 |       3.23e+01 |        3.05e+00 |      2.71e+01 |
|        4.74e+08 |       3.23e+01 |        3.23e+00 |      2.80e+01 |
|        5.61e+08 |       3.25e+01 |        2.71e+00 |      2.82e+01 |
|        3.10e+08 |       3.29e+01 |        4.32e+00 |      2.79e+01 |
|        3.17e+08 |       3.18e+01 |        2.69e+00 |      2.75e+01 |
|        7.32e+07 |       3.24e+01 |        2.65e+00 |      2.73e+01 |
+-----------------+----------------+-----------------+---------------+

+---------------+-------------+---------------+---------------+----------------+
| cov_stats.max | cov_stats.n | cov_stats.sum | sv_stats.mean | sv_stats.stdev |
+---------------+-------------+---------------+---------------+----------------+
|       float64 |       int64 |       float64 |       float64 |        float64 |
+---------------+-------------+---------------+---------------+----------------+
|      4.20e+01 |         114 |      3.62e+03 |      9.44e+03 |       2.95e+02 |
|      5.21e+01 |      

# 6. Format Table for Exporting

In this section, we format the table before exporting so it is in a usable format once written out. Specifically, we are flattening the table. This is done so that when the table is written out, the data is easier to work with. If the tables were written out without flattening them, the new annotated information would be in a nested structure which would make it difficult to work with outside of Hail. 

<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
    
<br>
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.flatten"> More on  <i> flatten() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.key_by"> More on <i>key_by()</i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.describe"> More on <i>describe()</i></a></li>
    
</details>

[Back to Index](#Index)

In [40]:
# Flatten out the structs created from annotating the tables
table = table.flatten()

# Change the keys of the table so that it is keyed by genetic region and population
table = table.key_by(table['hgdp_tgp_meta.Genetic.region'], table.population)

In [41]:
# Check format of the flattened table
table.show()

2023-11-30 22:12:00.120 Hail: INFO: Coerced sorted dataset
2023-11-30 22:12:00.783 Hail: INFO: Coerced sorted dataset
2023-11-30 22:12:01.401 Hail: INFO: Coerced sorted dataset
2023-11-30 22:12:02.429 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,,,,,,,,,,,,,,,,,,,,,,,
hgdp_tgp_meta.Genetic.region,population,n_snp_stats.mean,n_snp_stats.stdev,n_snp_stats.min,n_snp_stats.max,n_snp_stats.n,n_snp_stats.sum,cov_stats.mean,cov_stats.stdev,cov_stats.min,cov_stats.max,cov_stats.n,cov_stats.sum,sv_stats.mean,sv_stats.stdev,sv_stats.min,sv_stats.max,sv_stats.n,sv_stats.sum,cnv_stats.mean,cnv_stats.stdev,cnv_stats.min,cnv_stats.max,cnv_stats.n,cnv_stats.sum,n_unrelated
str,str,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,int64
"""AFR""","""ACB""",3.14e+06,4.63e+04,2.92e+06,3.20e+06,114,3.58e+08,3.17e+01,2.48e+00,2.84e+01,4.20e+01,114,3.62e+03,9.44e+03,2.95e+02,8.44e+03,1.01e+04,114,1.08e+06,5.03e+02,1.81e+01,4.59e+02,5.50e+02,114,5.74e+04,94
"""AFR""","""ASW""",3.08e+06,6.23e+04,2.88e+06,3.18e+06,71,2.18e+08,3.23e+01,3.36e+00,2.75e+01,5.21e+01,71,2.29e+03,9.22e+03,3.03e+02,8.43e+03,9.72e+03,71,6.54e+05,4.91e+02,1.90e+01,4.44e+02,5.29e+02,71,3.49e+04,52
"""AFR""","""BantuKenya""",3.18e+06,2.01e+04,3.14e+06,3.21e+06,12,3.82e+07,3.19e+01,3.88e+00,2.91e+01,4.40e+01,12,3.83e+02,8.43e+03,9.96e+01,8.29e+03,8.59e+03,12,1.01e+05,4.67e+02,3.61e+01,3.91e+02,5.31e+02,12,5.61e+03,10
"""AFR""","""BantuSouthAfrica""",3.27e+06,4.15e+04,3.21e+06,3.35e+06,8,2.62e+07,3.89e+01,1.03e+01,3.05e+01,6.45e+01,8,3.11e+02,8.59e+03,2.35e+02,8.15e+03,8.94e+03,8,6.87e+04,4.46e+02,3.64e+01,4.01e+02,4.97e+02,8,3.56e+03,8
"""AFR""","""Biaka""",3.41e+06,9.15e+03,3.39e+06,3.43e+06,26,8.86e+07,3.23e+01,3.05e+00,2.71e+01,4.01e+01,26,8.41e+02,8.79e+03,4.52e+02,7.95e+03,9.82e+03,26,2.29e+05,4.77e+02,2.48e+01,4.04e+02,5.24e+02,26,1.24e+04,22
"""AFR""","""ESN""",3.20e+06,8.46e+03,3.18e+06,3.23e+06,148,4.74e+08,3.23e+01,3.23e+00,2.80e+01,5.35e+01,148,4.78e+03,9.40e+03,3.63e+02,8.48e+03,9.94e+03,148,1.39e+06,5.03e+02,2.05e+01,4.41e+02,5.60e+02,148,7.44e+04,103
"""AFR""","""GWD""",3.19e+06,1.19e+04,3.15e+06,3.22e+06,176,5.61e+08,3.25e+01,2.71e+00,2.82e+01,4.34e+01,176,5.72e+03,9.42e+03,2.88e+02,8.58e+03,9.90e+03,176,1.66e+06,5.02e+02,1.86e+01,4.56e+02,5.43e+02,176,8.84e+04,116
"""AFR""","""LWK""",3.19e+06,9.91e+03,3.15e+06,3.22e+06,97,3.10e+08,3.29e+01,4.32e+00,2.79e+01,6.02e+01,97,3.19e+03,9.42e+03,2.80e+02,8.71e+03,9.87e+03,97,9.14e+05,5.01e+02,1.69e+01,4.38e+02,5.46e+02,97,4.86e+04,91


In [42]:
# Check the description of the table after flattening to make sure it is what we'd expect
table.describe()

----------------------------------------
Global fields:
    'global_annotation_descriptions': struct {
        gnomad_sex_imputation_ploidy_cutoffs: struct {
            Description: str
        }, 
        gnomad_population_inference_pca_metrics: struct {
            Description: str
        }, 
        sample_hard_filter_cutoffs: struct {
            Description: str
        }, 
        gnomad_sample_qc_metric_outlier_cutoffs: struct {
            Description: str
        }, 
        gnomad_age_distribution: struct {
            Description: str, 
            sub_globals: struct {
                bin_edges: struct {
                    Description: str
                }, 
                bin_freq: struct {
                    Description: str
                }, 
                n_smaller: struct {
                    Description: str
                }, 
                n_larger: struct {
                    Description: str
                }
            }
        }, 
        hgdp_tgp

In [43]:
# One last validity check before writing out the dataset is to make sure we still have the number of rows we'd expect
# In this case, since the data is grouped by genetic region, population, the number of rows should be equal to the number of populations (80)
table.count()

2023-11-30 22:12:31.914 Hail: INFO: Ordering unsorted dataset with network shuffle


80

# 7. Export Final Table
<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<br>
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.export"> More on  <i> export() </i></a></li>

</details>

[Back to Index](#Index)

- Write out the final table in tsv format 

```python3
table.export(final_table_path, header=True)
```

[Back to Index](#Index)